<div class="alert alert-block alert-info">
<h3>Student(s) Information</h3> Please provide information about yourself.<br>
<b>Name1</b>: Nabihah Rashid <br>
<b>Name2 (optional) </b>: Maryam Syed <br>
<b>NetID 1</b>: nr560 <br>
<b>NetID 2 (optional)</b>: ms2974<br>
<b>Notes to Grader</b> (optional):<br>
<br> <br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:      


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

### CS 439 - Introduction to Data Science
### Fall 2021

# Midsemester Project : Twitter Data Analysis

### Due Date : November 14th, 2021 by 11:59 PM


### About this project
This is your mid-semester project. You are allowed to work with a partner (if you'd like). Only one partner (or max two per group is allowed). The goal of this mid-semester project is to work with Twitter API to analyze tweets from a person, and in this case, President Donald Trump.  @RealDonaldTrump tweets provide a great opportunity to understand how online media can be used to communicate over the traditional media. Moreoover, Trump tweets has become so consequential, they actually can move the stock market on short term and get network TV to debate and discuss hours and hours about what Trump meant.

We hope this project will be fun as we can analyze range of emotions, hope, controversy, vagueness that are part of Trump tweets. We are interested in seeing what conclusions you can draw from US Presidents tweets. 
* DISCLAIMER: This project is not designed with any bias in mind. Note that we could pick either candiadate (Hillary Clinton or Donald Trump) or anyone else to do the same analysis. We hope your analysis is objective, independent of any political bias you may have. As Data Scientists, it is our responsiblity to do independent analysis of the data we try to understand. You should follow data and interpret w/o any bias.


## Set up
Let us get all the libaries initialized as necessary

In [1]:
# Run this cell to set up your notebook
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import json

# Ensure that Pandas shows at least 280 characters in columns, so we can see full tweets
pd.set_option('max_colwidth', 280)

%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re

## Downloading Recent Tweets
It is important to download the most recent tweets (especially if you are working as a group). Those who are working by themselves are allowed to use the downloaded files w/o setting up access to any twitter API (which can sometime be bit complicated). Twitter provides the API Tweepy (http://www.tweepy.org/) that makes it easy to access twitter content that is publicly available. We will also provide example code as needed.

In [2]:
!pip install tweepy==4.2.0

In [3]:
## Make sure you have set up tweepy if you are working locally.
# https://www.pythoncentral.io/introduction-to-tweepy-twitter-for-python/
# After set up, the following should run:
import tweepy

## PART 1:  Accessing Twitter API  (optional for individuals)
In order to access Twitter API, you need to get keys by signing up as a Twitter developer. We will walk you through this process. 
* if you are working by yourself on this project, you can skip PART 1, and complete the project using the data files provided in the data folder. PART 1 is optional for those working by themselves. However, we highly recommend that you do Part 1 (after completing the project with offline data) if you would like to "learn" how to use Twitter API that might be useful.

### Task 1.1

Follow the instructions below to get your Twitter API keys.  **Read the instructions completely before starting.**

1. [Create a Twitter account](https://twitter.com/).  You can use an existing account if you have one; if you prefer to not do this assignment under your regular account, feel free to create a throw-away account.
2. Under account settings, add your phone number to the account.
3. [Create a Twitter developer account](https://developer.twitter.com/en/apply/) by clicking the 'Apply' button on the top right of the page. Attach it to your Twitter account. You'll have to fill out a form describing what you want to do with the developer account. Explain that you are doing this for a class at Rutgers University and that you don't know exactly what you're building yet and just need the account to get started. These applications are approved by some sort of AI system, so it doesn't matter exactly what you write. Just don't enter a bunch of alweiofalwiuhflawiuehflawuihflaiwhfe type stuff or you might get rejected.
4. Once you're logged into your developer account, [create an application for this assignment](https://apps.twitter.com/app/new).  You can call it whatever you want, and you can write any URL when it asks for a web site.  You don't need to provide a callback URL.
5. On the page for that application, find your Consumer Key and Consumer Secret.
6. On the same page, create an Access Token.  Record the resulting Access Token and Access Token Secret.
7. Edit the file [keys.json](keys.json) and replace the placeholders with your keys.

## WARNING (Please Read) !!!!


### Protect your Twitter Keys
<span style="color:red">
If someone has your authentication keys, they can access your Twitter account and post as you!  So don't give them to anyone, and **don't write them down in this notebook**. 
</span>
The usual way to store sensitive information like this is to put it in a separate file and read it programmatically.  That way, you can share the rest of your code without sharing your keys.  That's why we're asking you to put your keys in `keys.json` for this assignment.


### Avoid making too many API calls.

<span style="color:red">
Twitter limits developers to a certain rate of requests for data.  If you make too many requests in a short period of time, you'll have to wait awhile (around 15 minutes) before you can make more.  </span> 
So carefully follow the code examples you see and don't rerun cells without thinking.  Instead, always save the data you've collected to a file.  We've provided templates to help you do that.


### Be careful about which functions you call!

<span style="color:red">
This API can retweet tweets, follow and unfollow people, and modify your twitter settings.  Be careful which functions you invoke! </span> It is possible that you can accidentally re-tweet some tweets because you typed `retweet` instead of `retweet_count`. 
</span>


In [4]:
import json
key_file = 'keys.json'
# Loading your keys from keys.json (which you should have filled
# in in question 1):
with open(key_file) as f:
    keys = json.load(f)
# if you print or view the contents of keys be sure to delete the cell!

### Task 1.2 Testing Twitter Authentication
This following code should run w/o erros or warnings and display Rutgers University's twitter username

In [5]:
import tweepy
from tweepy import TweepyException
import logging

try:
    auth = tweepy.OAuthHandler(keys["consumer_key"], keys["consumer_secret"])
    redirect_url = auth.get_authorization_url()
    auth.set_access_token(keys["access_token"], keys["access_token_secret"])
    api = tweepy.API(auth)
    print("Rutgers username is:", api.get_user(screen_name="RutgersU").name)
except TweepyException as e:
    logging.warning("There was a Tweepy error. Double check your API keys and try again.")
    logging.warning(e)

## PART 2 - Working with Twitter
The json file in data folder contains (to be downloaded by you) some loaded tweets from @RutgersU. Run it and read the code. You can also try other json files in the data folder to try this. 

In [6]:
from pathlib import Path
import json

ds_tweets_save_path = "data/RutgersU_recent_tweets.json"   # need to get this file

# Guarding against attempts to download the data multiple
# times:
if not Path(ds_tweets_save_path).is_file():
    # Getting as many recent tweets by @RutgersU as Twitter will let us have.
    # We use tweet_mode='extended' so that Twitter gives us full 280 character tweets.
    # This was a change introduced in September 2017.
    
    # The tweepy Cursor API actually returns "sophisticated" Status objects but we 
    # will use the basic Python dictionaries stored in the _json field. 
    example_tweets = [t._json for t in tweepy.Cursor(api.user_timeline, screen_name="RutgersU", 
                                             tweet_mode='extended').items()]
    
    # Saving the tweets to a json file on disk for future analysis
    with open(ds_tweets_save_path, "w") as f:        
        json.dump(example_tweets, f)

# Re-loading the json file:
with open(ds_tweets_save_path, "r") as f:
    example_tweets = json.load(f)

If things ran as expected, you should be able to look at the first tweet by running the code below. It probabably does not make sense to view all tweets in a notebook, as size of the tweets can freeze your browser (always a good idea to press ctrl-S to save the latest, in case you have to restart Jupyter)

In [7]:
# Looking at one tweet object, which has type Status: 
from pprint import pprint # ...to get a more easily-readable view.
print(example_tweets[0])

### Task 2.1 (Optional for Individuals)

### What you need to do. 

Re-factor the above code fragment into reusable snippets below.  You should not need to make major modifications; this is mostly an exercise in understanding the above code block. 

In [8]:
def load_keys(path):
    """Loads your Twitter authentication keys from a file on disk.
    
    Args:
        path (str): The path to your key file.  The file should
          be in JSON format and look like this (but filled in):
            {
                "consumer_key": "<your Consumer Key here>",
                "consumer_secret":  "<your Consumer Secret here>",
                "access_token": "<your Access Token here>",
                "access_token_secret": "<your Access Token Secret here>"
            }
    
    Returns:
        dict: A dictionary mapping key names (like "consumer_key") to
          key values."""
    
    ### BEGIN SOLUTION
    if not Path(path).is_file(): 
      example_tweets = [t._json for t in tweepy.Cursor(api.user_timeline, screen_name="RutgersU", 
                                             tweet_mode='extended').items()]
    
    # Saving the tweets to a json file on disk for future analysis
      with open(path, "w") as f:        
        json.dump(path, f)

    # Re-loading the json file:
    with open(path, "r") as f:
      example_tweets = json.load(f)
    
    return example_tweets

    ### END SOLUTION

In [9]:
def download_recent_tweets_by_user(user_account_name, keys):
    """Downloads tweets by one Twitter user.

    Args:
        user_account_name (str): The name of the Twitter account
          whose tweets will be downloaded.
        keys (dict): A Python dictionary with Twitter authentication
          keys (strings), like this (but filled in):
            {
                "consumer_key": "<your Consumer Key here>",
                "consumer_secret":  "<your Consumer Secret here>",
                "access_token": "<your Access Token here>",
                "access_token_secret": "<your Access Token Secret here>"
            }

    Returns:
        list: A list of Dictonary objects, each representing one tweet."""
    import tweepy
    
    ### BEGIN SOLUTION 
    auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
    auth.set_access_token(keys['access_token'], keys['access_token_secret'])

    api = tweepy.API(auth)

    tweet_list=[]
    tweets = api.user_timeline(screen_name=user_account_name)
    
    for tweet in tweets:
      tweet_list.append(tweet)
  
    return tweet_list 
    ### END SOLUTION

In [10]:
def load_tweets(path):
    """Loads tweets that have previously been saved.
    
    Calling load_tweets(path) after save_tweets(tweets, path)
    will produce the same list of tweets.
    
    Args:
        path (str): The place where the tweets were be saved.

    Returns:
        list: A list of Dictionary objects, each representing one tweet."""
    
    ### BEGIN SOLUTION
    with open(path, "rb") as f:
        import json
        return json.load(f)
    
    ### END SOLUTION

In [11]:
def save_tweets(tweets, path):
    with open(path, "w") as f:     
        json.dump(tweets, f)

In [12]:
def get_tweets_with_cache(user_account_name, keys_path):
    """Get recent tweets from one user, loading from a disk cache if available.
    
    The first time you call this function, it will download tweets by
    a user.  Subsequent calls will not re-download the tweets; instead
    they'll load the tweets from a save file in your local filesystem.
    All this is done using the functions you defined in the previous cell.
    This has benefits and drawbacks that often appear when you cache data:
    
    +: Using this function will prevent extraneous usage of the Twitter API.
    +: You will get your data much faster after the first time it's called.
    -: If you really want to re-download the tweets (say, to get newer ones,
       or because you screwed up something in the previous cell and your
       tweets aren't what you wanted), you'll have to find the save file
       (which will look like <something>_recent_tweets.pkl) and delete it.
    
    Args:
        user_account_name (str): The Twitter handle of a user, without the @.
        keys_path (str): The path to a JSON keys file in your filesystem.
    """
    
    ### BEGIN SOLUTION
    print("in cache")
    if not Path(user_account_name + '_recent_tweets.pkl').is_file():
        tweets = download_recent_tweets_by_user(user_account_name, load_keys(keys_path))
        save_tweets(tweets, user_account_name + '_recent_tweets.pkl')
        return tweets
    return load_tweets(user_account_name + '_recent_tweets.pkl')
    
    ### END SOLUTION

If everything was implemented correctly you should be able to obtain roughly the last 3000 tweets by @RutgersU. (This may take a few minutes)

In [13]:
# When you are done, run this cell to load @RutgersU 's tweets.
# Note the function get_tweets_with_cache.  You may find it useful
# later.
# rutgers_tweets = get_tweets_with_cache("RutgersU", "keys1.json")
# print("Number of tweets downloaded:", len(rutgers_tweets))

### Task 2.2
To be consistent we are going to use the same dataset no matter what you get from your twitter api. So from this point on, if you are working as a group or individually, be sure to use the data sets provided to you in the zip file. There should be two json files inside your data folder. One is '2017-2018.json', the other one is '2016-2017.json'. We will load the '2017-2018.json' first.

In [14]:
dest_path = "data/2017-2018.json"
trump_tweets = load_tweets(dest_path)

If everything is working correctly correctly this should load roughly the last 3000 tweets by `realdonaldtrump`.

In [15]:
assert 2000 <= len(trump_tweets) <= 4000

If the assert statement above works, then continue on to task 2.3.

### Task 2.3

Find the number of the month of the oldest tweet.

In [16]:
# Enter the number of the month of the oldest tweet (e.g. 1 for January)
oldest_month = 10


trump_tweets = pd.DataFrame(trump_tweets)
### BEGIN SOLUTION

oldest = trump_tweets.iloc[[len(trump_tweets)-1]][['created_at']]
oldest = pd.to_datetime(oldest['created_at'].to_numpy())
month = oldest.month
print(month)

### END SOLUTION

## PART 3  Twitter Source Analysis



### Task 3.1

Create a new data frame from `2016-2017.json` and merge with `trump_tweets` 

**Important:** There may/will be some overlap so be sure to __eliminate duplicate tweets__. If you do not eliminate the duplicates properly, your results might not be compatible with the test solution. 
**Hint:** the `id` of a tweet is always unique.

In [17]:
# if you do not have new tweets, then all_tweets is the same as  old_trump_tweets
### BEGIN SOLUTION

dest_path_2 = "data/2016-2017.json"
trump_tweets_2 = load_tweets(dest_path_2)
trump_tweets_2 = pd.DataFrame(trump_tweets_2)

trump_tweets_changed = trump_tweets[['id', 'source', 'full_text', 'retweet_count']]
trump_tweets_changed_2 = trump_tweets_2[['id', 'source', 'text', 'retweet_count']]

trump_tweets_changed = trump_tweets_changed.rename(columns={'full_text': 'text'})
trump_tweets_changed['time'] = pd.to_datetime(trump_tweets['created_at'].to_numpy())
trump_tweets_changed_2['time'] = pd.to_datetime(trump_tweets_2['created_at'].to_numpy())

trump_tweets_changed.loc[:, 'id'] = trump_tweets_changed['id'].astype('int64')
trump_tweets_changed_2.loc[:, 'id'] = trump_tweets_changed_2['id'].astype('int64')

trump_tweets_changed = trump_tweets_changed.set_index('id')
trump_tweets_changed_2 = trump_tweets_changed_2.set_index('id')


In [18]:
# trump_tweets_2["id"] = pd.to_numeric(trump_tweets_2["id"])
# trump_tweets_2.rename(columns={'text': 'full_text'}, inplace =True)

# # get cols for trump_tweets
# trump_tweets_changed = pd.DataFrame(index = trump_tweets.index)
# trump_tweets_changed['time'] = pd.to_datetime(trump_tweets['created_at'].to_numpy())
# trump_tweets_changed['source'] = trump_tweets['source'].to_numpy()
# trump_tweets_changed['text'] = trump_tweets['full_text'].to_numpy()
# trump_tweets_changed['retweet_count'] = trump_tweets['retweet_count'].to_numpy()

# #get cols for trump_tweets2
# trump_tweets_changed_2 = pd.DataFrame(index = trump_tweets_2.index)
# trump_tweets_changed_2['time'] = pd.to_datetime(trump_tweets_2['created_at'].to_numpy())
# trump_tweets_changed_2['source'] = trump_tweets_2['source'].to_numpy()
# trump_tweets_changed_2['text'] = trump_tweets_2['full_text'].to_numpy()
# trump_tweets_changed_2['retweet_count'] = trump_tweets_2['retweet_count'].to_numpy()

In [19]:
# if you do not have new tweets, then all_tweets is the same as  old_trump_tweets
all_tweets = pd.concat([trump_tweets_changed, trump_tweets_changed_2])

print('all_tweets size before dropping dups:', all_tweets.size)
# all_tweets = all_tweets[~all_tweets.index.duplicated(keep='first')]
all_tweets = all_tweets.drop_duplicates(keep ='first')

print(trump_tweets_changed.size)
print(trump_tweets_changed_2.size)

print('all_tweets size after dropping dups:', all_tweets.size)
all_tweets.sort_index()

### END SOLUTION

assert(all_tweets.size == 40176) 



In [20]:
all_tweets.columns

### Task 3.2
Construct a DataFrame called `df_trump` containing all the tweets stored in `all_tweets`. The index of the dataframe should be the ID of each tweet (looks something like `907698529606541312`). It should have these columns:

- `time`: The time the tweet was created encoded as a datetime object. (Use `pd.to_datetime` to encode the timestamp.)
- `source`: The source device of the tweet.
- `text`: The text of the tweet.
- `retweet_count`: The retweet count of the tweet. 

Finally, **the resulting dataframe should be sorted by the index.**

**Warning:** *Some tweets will store the text in the `text` field and other will use the `full_text` field.*

**Warning:** *Don't forget to check the type of index*

In [21]:
### BEGIN SOLUTION
df_trump = all_tweets
df_trump.sort_index()
df_trump
### END SOLUTION

In the following questions, we are going to find out the charateristics of Trump tweets and the devices used for the tweets.

First let's examine the source field:

In [22]:
df_trump['source'].unique()

## Task 3.3

Remove the HTML tags from the source field. 

**Hint:** Use `df_trump['source'].str.replace` and your favorite regular expression.

In [23]:
### BEGIN SOLUTION
import re

df_trump['source']= df_trump['source'].str.replace('<[^>]*>', '')

### END SOLUTION

### Make a plot to find out the most common device types used in accessing twitter

Sort the plot in decreasing order of the most common device type

In [24]:
### BEGIN SOLUTION
labels = ['Twitter for iPhone','Twitter for Android','Twitter Web Client','Media Studio',
'Twitter Ads',
'Twitter for iPad',
'Instagram',
'TweetDeck',
'Periscope',
'Mobile Web (M5)']
ax = sns.countplot(data = df_trump, x = 'source', order = df_trump['source'].value_counts().index)
ax.set_xticklabels(labels, rotation=90 )
### END SOLUTION

### Task 3.4
Is there a difference between his Tweet behavior across these devices? We will attempt to answer this question in our subsequent analysis.

First, we'll take a look at whether Trump's tweets from an Android come at different times than his tweets from an iPhone. Note that Twitter gives us his tweets in the [UTC timezone](https://www.wikiwand.com/en/List_of_UTC_time_offsets) (notice the `+0000` in the first few tweets)

**Note** - If your `time` column is not in datetime format, the following code will not work.

In [25]:
df_trump['time'][0:3]

We'll convert the tweet times to US Eastern Time, the timezone of New York and Washington D.C., since those are the places we would expect the most tweet activity from Trump.

In [26]:
df_trump['est_time'] = (
    df_trump['time'] # Set initial timezone to UTC
                 .dt.tz_convert("EST") # Convert to Eastern Time
)
df_trump.head()

**What you need to do:**

Add a column called `hour` to the `df_trump` table which contains the hour of the day as floating point number computed by:

$$
\text{hour} + \frac{\text{minute}}{60} + \frac{\text{second}}{60^2}
$$

In [27]:
from datetime import datetime
new = df_trump['est_time'].reset_index()
for i in range(len(new)):
    dateTimeObj = new.loc[i,'est_time']
    date_str = dateTimeObj.strftime("%Y-%m-%d %H:%M:%S")
    time = date_str.split()[1].split(":")
    # print(time)
    hour = float(time[0])
    minute = float(time[1])/60
    second = float(time[2])/3600
    
    new.loc[i, 'hour'] = hour + minute + second


In [28]:
df_trump["hour"] = new["hour"].to_numpy()
df_trump['roundhour']= round(df_trump['hour'])

In [29]:
assert np.isclose(df_trump.loc[690171032150237184]['hour'], 8.93639)

Use the `roundhour` column and plot the number of tweets at every hour of the day.
Order the plot using the hour of the day (1 to 24). Use seaborn `countplot`

In [30]:
# make a bar plot here
### BEGIN SOLUTION
df_trump['roundhour'] = df_trump['roundhour'].replace([0],24) 
hour = np.arange(1, 25)
ax = sns.countplot(data = df_trump, x = 'roundhour', order = hour)
ax.set_xticklabels(hour, rotation=90)

### END SOLUTION

Now, use this data along with the seaborn `distplot` function to examine the distribution over hours of the day in eastern time that trump tweets on each device for the 2 most commonly used devices.  Your plot should look somewhat similar to the following. 
![device_hour2.png](attachment:device_hour2.png)


In [31]:
### BEGIN SOLUTION

android = df_trump.groupby(['source']).get_group('Twitter for Android')['roundhour']
iphone = df_trump.groupby(['source']).get_group('Twitter for iPhone')['roundhour']
ax = sns.distplot(iphone, kde = True, hist = False, color = 'b')
ax = sns.distplot(android, kde = True, hist = False, color = 'r')
plt.legend(labels=['iPhone', 'Android'])
### END SOLUTION

### Task 3.5

According to [this Verge article](https://www.theverge.com/2017/3/29/15103504/donald-trump-iphone-using-switched-android), Donald Trump switched from an Android to an iPhone sometime in March 2017.

Create a figure identical to your figure from 3.4, except that you should show the results only from 2016. If you get stuck consider looking at the `year_fraction` function from the next problem.

Use this data along with the seaborn `distplot` function to examine the distribution over hours of the day in eastern time that trump tweets on each device for the 2 most commonly used devices.  Your plot should look somewhat similar to the following. 

During the campaign, it was theorized that Donald Trump's tweets from Android were written by him personally, and the tweets from iPhone were from his staff. Does your figure give support the theory?

Response: In 2016, the time allocation for the usage of the iphone centered in the afternoon, while his tweets from 2015 to present shows that he mostly tweets in the morning. It seems that the tweets from iphone in 2016 were from his staff, not himself.

![title](images/device_hour2.png)

In [32]:
import datetime
def year_fraction(date):
    start = datetime.date(date.year, 1, 1).toordinal()
    year_length = datetime.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length

df_trump['year'] = df_trump['time'].apply(year_fraction) #should be df_trump

In [33]:
### BEGIN SOLUTION
df_trump['year'] = df_trump['time'].apply(year_fraction) 
df_trump['roundyear']=(df_trump['year'].astype(int))
df_trump_2016 = df_trump.groupby(['roundyear']).get_group(2016)

android = df_trump_2016.groupby(['source']).get_group('Twitter for Android')['roundhour']
iphone = df_trump_2016.groupby(['source']).get_group('Twitter for iPhone')['roundhour']

ax = sns.distplot(iphone, kde = True, hist = False, color = 'b')
ax = sns.distplot(android, kde = True, hist = False, color = 'r')

plt.legend(labels=['iPhone', 'Android'])

### END SOLUTION

### Task 3.6
Edit this cell to answer the following questions.
* What time of the day the Android tweets were made by Trump himself?
    morning
* What time of the day the Android tweets were made by paid staff?
    afternoon/mid-day


Note that these are speculations based on what you observe in the data set.

### Task 3.7 Device Analysis
Let's now look at which device he has used over the entire time period of this dataset.

To examine the distribution of dates we will convert the date to a fractional year that can be plotted as a distribution.

(Code borrowed from https://stackoverflow.com/questions/6451655/python-how-to-convert-datetime-dates-to-decimal-years)

In [34]:
import datetime
def year_fraction(date):
    start = datetime.date(date.year, 1, 1).toordinal()
    year_length = datetime.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length


df_trump['year'] = df_trump['time'].apply(year_fraction) #should be df_trump

Use the `sns.distplot` to overlay the distributions of the 2 most frequently used web technologies over the years.  Your final plot should be similar to:

![source_years.png](attachment:source_years.png)

In [35]:
### BEGIN SOLUTION

android1 = df_trump.groupby(['source']).get_group('Twitter for Android')['year']
iphone1 = df_trump.groupby(['source']).get_group('Twitter for iPhone')['year']

ax = sns.distplot(iphone1, kde = True)
ax = sns.distplot(android1, kde = True)

plt.legend(labels=['iPhone', 'Android'])

### END SOLUTION

## PART 4 - Sentiment Analysis

It turns out that we can use the words in Trump's tweets to calculate a measure of the sentiment of the tweet. For example, the sentence "I love America!" has positive sentiment, whereas the sentence "I hate taxes!" has a negative sentiment. In addition, some words have stronger positive / negative sentiment than others: "I love America." is more positive than "I like America."

We will use the [VADER (Valence Aware Dictionary and sEntiment Reasoner)](https://github.com/cjhutto/vaderSentiment) lexicon to analyze the sentiment of Trump's tweets. VADER is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media which is great for our usage.

The VADER lexicon gives the sentiment of individual words. Run the following cell to show the first few rows of the lexicon:

In [36]:
print(''.join(open("data/vader_lexicon.txt").readlines()[:10]))

### Task 4.1

As you can see, the lexicon contains emojis too! The first column of the lexicon is the *token*, or the word itself. The second column is the *polarity* of the word, or how positive / negative it is.

(How did they decide the polarities of these words? What are the other two columns in the lexicon? See the link above.)

 Read in the lexicon into a DataFrame called `df_sent`. The index of the DF should be the tokens in the lexicon. `df_sent` should have one column: `polarity`: The polarity of each token.

In [37]:
### BEGIN SOLUTION
df_sent = pd.read_csv('data/vader_lexicon.txt', header = None, sep="\t", index_col = 0)
df_sent.drop(axis = 'columns', labels = [2, 3], inplace=True)
df_sent.rename(columns ={1:'polarity'}, inplace=True)
df_sent

### END SOLUTION

### Task 4.2

Now, let's use this lexicon to calculate the overall sentiment for each of Trump's tweets. Here's the basic idea:

1. For each tweet, find the sentiment of each word.
2. Calculate the sentiment of each tweet by taking the sum of the sentiments of its words.

First, let's lowercase the text in the tweets since the lexicon is also lowercase. Set the `text` column of the `df_trump` DF to be the lowercased text of each tweet.

In [38]:
### BEGIN SOLUTION
df_trump['text'] = df_trump['text'].str.lower()
### END SOLUTION

### Task 4.3

Now, let's get rid of punctuation since it'll cause us to fail to match words. Create a new column called `no_punc` in the `df_trump` to be the lowercased text of each tweet with all punctuation replaced by a single space. We consider punctuation characters to be any character that isn't a Unicode word character or a whitespace character. You may want to consult the Python documentation on regexes for this problem.

(Why don't we simply remove punctuation instead of replacing with a space? See if you can figure this out by looking at the tweet data.)

In [39]:
# Save your regex in punct_re
import re
punct_re = r'[^\w\s\n]'

### BEGIN SOLUTION
regex = re.compile(punct_re)
df_trump['no_punc'] = df_trump['text'].replace(to_replace=regex, value= ' ')

### END SOLUTION

In [40]:
assert isinstance(punct_re, str)
assert re.search(punct_re, 'this') is None
assert re.search(punct_re, 'this is ok') is None
assert re.search(punct_re, 'this is\nok') is None
assert re.search(punct_re, 'this is not ok.') is not None
assert re.search(punct_re, 'this#is#ok') is not None
assert re.search(punct_re, 'this^is ok') is not None
assert df_trump['no_punc'].loc[800329364986626048] == 'i watched parts of  nbcsnl saturday night live last night  it is a totally one sided  biased show   nothing funny at all  equal time for us '
assert df_trump['text'].loc[884740553040175104] == 'working hard to get the olympics for the united states (l.a.). stay tuned!'


### Task 4.4


Now, let's convert the tweets into what's called a [*tidy format*](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) to make the sentiments easier to calculate. Use the `no_punc` column of `df_trump` to create a table called `tidy_format`. The index of the table should be the IDs of the tweets, repeated once for every word in the tweet. It has two columns:

1. `num`: The location of the word in the tweet. For example, if the tweet was "i love america", then the location of the word "i" is 0, "love" is 1, and "america" is 2.
2. `word`: The individual words of each tweet.

The first few rows of our `tidy_format` table look like:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>num</th>
      <th>word</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>894661651760377856</th>
      <td>0</td>
      <td>i</td>
    </tr>
    <tr>
      <th>894661651760377856</th>
      <td>1</td>
      <td>think</td>
    </tr>
    <tr>
      <th>894661651760377856</th>
      <td>2</td>
      <td>senator</td>
    </tr>
    <tr>
      <th>894661651760377856</th>
      <td>3</td>
      <td>blumenthal</td>
    </tr>
    <tr>
      <th>894661651760377856</th>
      <td>4</td>
      <td>should</td>
    </tr>
  </tbody>
</table>

You can double check that your tweet with ID `894661651760377856` has the same rows as ours. Our tests don't check whether your table looks exactly like ours.

As usual, try to avoid using any for loops. Our solution uses a chain of 5 methods on the 'trump' DF, albeit using some rather advanced Pandas hacking.

* **Hint 1:** Try looking at the `expand` argument to pandas' `str.split`.

* **Hint 2:** Try looking at the `stack()` method.

* **Hint 3:** Try looking at the `level` parameter of the `reset_index` method.

In [41]:
#tidy_format = ...

### BEGIN SOLUTION
tidy_format = df_trump['no_punc'].str.split(expand=True).stack().to_frame().reset_index(level = 1)
tidy_format = tidy_format.rename(columns = {"level_1" : "num", 0 : 'word'})
tidy_format[:].loc[894661651760377856]

### END SOLUTION

In [42]:
assert tidy_format.loc[894661651760377856].shape == (27, 2)
assert ' '.join(list(tidy_format.loc[894661651760377856]['word'])) == 'i think senator blumenthal should take a nice long vacation in vietnam where he lied about his service so he can at least say he was there'

### Task 4.5

Now that we have this table in the tidy format, it becomes much easier to find the sentiment of each tweet: we can join the table with the lexicon table. 

Add a `polarity` column to the `df_trump` table.  The `polarity` column should contain the sum of the sentiment polarity of each word in the text of the tweet.

**Hint** you will need to merge the `tidy_format` and `df_sent` tables and group the final answer.


In [43]:
#df_trump['polarity'] = ...

### BEGIN SOLUTION
merged_df = tidy_format.merge(df_sent, left_on = 'word', right_index= True)
df_trump['polarity'] = merged_df['polarity'].reset_index().groupby('id').sum()
df_trump['polarity'] = df_trump['polarity'].fillna(0)
### END SOLUTION

In [44]:
assert np.allclose(df_trump.loc[744701872456536064, 'polarity'], 8.4)
assert np.allclose(df_trump.loc[745304731346702336, 'polarity'], 2.5)
assert np.allclose(df_trump.loc[744519497764184064, 'polarity'], 1.7)
assert np.allclose(df_trump.loc[894661651760377856, 'polarity'], 0.2)
assert np.allclose(df_trump.loc[894620077634592769, 'polarity'], 5.4)
# If you fail this test, you dropped tweets with 0 polarity
assert np.allclose(df_trump.loc[744355251365511169, 'polarity'], 0.0)


### Task 4.6
Now we have a measure of the sentiment of each of his tweets! You can read over the VADER readme to understand a more robust sentiment analysis.
Now, write the code to see the most positive and most negative tweets from Trump in your dataset:
Find the most negative and most positive tweets made by Trump

In [45]:
### BEGIN SOLUTION

print('Most negative tweets:')
sorted_negative = df_trump.sort_values('polarity')
sorted_negative.head()['text']

### END SOLUTION

In [46]:
### BEGIN SOLUTION

print('Most positive tweets:')
sorted_positive= df_trump.sort_values('polarity', ascending=False)
sorted_positive.head()['text']
    
### END SOLUTION

### Task 4.7
Plot the distribution of tweet sentiments broken down by whether the text of the tweet contains `nyt` or `fox`.  Then in the box below comment on what we observe?

![title](images/nyt_vs_fox.png)

In [47]:
### BEGIN SOLUTION
plt.figure(figsize = (10,5))
nyt = df_trump[df_trump['text'].str.contains("nyt")]
fox = df_trump[df_trump['text'].str.contains("fox")]
ax = sns.distplot(nyt['polarity'], kde=True)
ax = sns.distplot(fox['polarity'], kde = True)

plt.legend(labels=['nyt', 'fox'])

### END SOLUTION

##### Comment on what you observe:

#### BEGIN SOLUTION
The tweets that contain nyt have a more negative sentiment and the tweets that contain fox have a more positive sentiment

#### END SOLUTION

## PART 5 - Principal Component Analysis (PCA) and Twitter
A look at the top words used and the sentiments expressed in Trump tweets indicates that, some words are used with others almost all the time. A notable example is the slogan like Make America Great Again. As such, it may be beneficial to look at groups of words rather than individual words. For that, we will look at an approach applying a Principal Component Analysis. 

### The PCA
The Principal Component Analysis, or PCA, is a tool generally used to identify patterns and to reduce the number of variables you have to consider in your analysis. For example, if you have data with 200 columns, it may be that a significant amount of the variance in your data can be explained by just 100 principal components. In the PCA, the first component is chosen in such a way that has the largest variance, subsequent components are orthogonal and continue covering as much variance as possible. In this way, the PCA samples as much of the variability in the data set with the first few components. Mathematically, each component is a linear combination of all the input parameters times coefficients specific for that component. These coefficients, or loading factors, are constrained such that the sum of the squares of them are equal to 1. As such, the loading factors serve as weights describing how strongly certain parameters contribute to the specific principal component. Parameters with large values of positive or negative loading factors are correlated with each other, which can serve to identify trends in your data.

### Task 5.1 Cleaning up the Data
Using NLTK (Natural Language Toolkit) package for language processing and other python libraries, parse the json file to deal with inflected words, such as plurals, and removed stop words like common English words (the, and, it, etc) and certain political terms (the candidates names, for example). You can start with the top 50 words, but full analysis may require large number of words.
Create a document-frequecy (df) matrix with 5000 rows and 50 columns where each column is a particular word (feature) and each row is a tweet (observation). The values of the matrix is how often the word appears. Apply the techniques we learned to reduce the weight of most common words (if necessary). Since this is a sparse matrix, you can use the sparse martix libraries to make things a bit more efficient (we can also use a regular numpy arrays to store these things since the dimensions are not too large). Lecture 6.1 captures some sparse matrix routines you can use.
Print the first 10 rows of the df to show the matrix you created

Start with the `tidy_format` dataframe

In [49]:
### BEGIN SOLUTION
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

# without dropping dups, the .split() line of code in the for loop won't work
all_tweets = all_tweets[~all_tweets.index.duplicated(keep='first')]
df_trump1 = all_tweets

#create a dataframe called tmp to store all words appear in the tweets
stopwrd = stopwords.words('english')
stopwrd.extend(['rt','t','co','https','realdonaldtrump','amp',"u",'hillary','trump2016','trump','clinton','http','ha','wa'])
tmp = tidy_format

#remove stopwords  
# changes them to empty rows first
tmp = tmp[~tmp['word'].isin(stopwrd)]

#deal with plurals
lemmatizer = WordNetLemmatizer()
tmp['word'] = tmp.word.apply(lemmatizer.lemmatize)

# Remove numbers
tmp = tmp[~(tmp['word'].str.contains('[0-9]+'))]

#Remove words with only 1 or 2 length
tmp = tmp[~tmp['word'].apply(lambda x: len(x) <= 2)]
# tmp['word'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

# get the words with top 50 frequency
top50 = tmp['word'].value_counts().nlargest(50).to_frame()
#filter the ids
tmp2 = tmp[tmp['word'].isin(top50.index)]
#get first 5000 unique ids
idlist = tmp2.index.unique()
idlist = idlist[:5000]
#create matrix
matrix = np.zeros((5000, 50))
words = top50.index
for i in range(5000):
    for j in range(50):
        if (words[j]) in (df_trump1['text'].loc[idlist[i]]).split():
            matrix[i][j] += 1
### END SOLUTION


### Task 5.2 Find the PCA's
Write the code to find the first 50 PCA's for the document-frequency matrix. Pass the document-term-matrix to scikit-learn’s (https://scikit-learn.org/stable/modules/decomposition.html#decompositions) PCA method to obtain the components and loading factors.

In [50]:
### BEGIN SOLUTION
from sklearn.decomposition import PCA

#calculate SVD
mu = matrix.mean(axis = 0)
matrix_ = matrix - mu

Sigma = matrix_.T @ matrix / len(matrix_)

import scipy.linalg as linalg
u, lam, uh = linalg.svd(Sigma)

pca = PCA(n_components = 50)
pca.fit(matrix)

### END SOLUTION

### Task 5.3 Examine the PCA
We can examine the PCA results to look at the heatmap. Make a grid plot which shows the various principal component along the x-axis and the individual words along the y-axes. Each grid box should be color-coded based on the sign of the loading factor and how large the square of that value is. Looking at it vertically, you can see which words constitute your principal components. Looking at it horizontally, you can see how individual terms are shared between components. 

![title](images/pca.png)



In [51]:
### BEGIN SOLUTION

fig = plt.figure( figsize = (30,30))
color = sns.diverging_palette(100, 400, as_cmap=True)
ax = sns.heatmap(pca.components_, cmap = color,
                 xticklabels=["PCA"+str(x) for x in range(1,pca.n_components_+1)], yticklabels = words)
ax.set_aspect("equal")

### END SOLUTION

### Task 5.4 PCA Compare
We can determine how many words and how many components are needed to do a good visualization. Plot PC1 and PC2 in a 2D plot. The results should be similar to following scatter plot 

![title](images/PC1_PC2.png)

This is a scatter plot of the values of the components, but with arrows indicating some of the prominent terms as indicated by their loading factors. The values of the loading factors are used to determine the length and direction of these arrows and as such they serve as a way of expressing direction. That is, tweets which use these terms will be moved along the length of those arrows. Shown are the most important parameters.

In [52]:
### BEGIN SOLUTION
pca0 = u[:,0]
pca1 = u[:,1]

pca0= (pca0 - pca0.mean())/ (pca0.max() - pca0.min())
pca1= (pca1 - pca1.mean())/ (pca1.max() - pca1.min())

sns.jointplot(pca0, pca1)

### END SOLUTION

## PART 6 - Twitter Engagement

In this problem, we'll explore which words led to a greater average number of retweets. For example, at the time of this writing, Donald Trump has two tweets that contain the word 'oakland' (tweets 932570628451954688 and 1016609920031117312) with 36757 and 10286 retweets respectively, for an average of 23,521.5.


Your `top_20` table should have this format:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>retweet_count</th>
    </tr>
    <tr>
      <th>word</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>jong</th>
      <td>40675.666667</td>
    </tr>
    <tr>
      <th>try</th>
      <td>33937.800000</td>
    </tr>
    <tr>
      <th>kim</th>
      <td>32849.595745</td>
    </tr>
    <tr>
      <th>un</th>
      <td>32741.731707</td>
    </tr>
    <tr>
      <th>maybe</th>
      <td>30473.192308</td>
    </tr>
  </tbody>
</table>

### Task 6.1
Find the top 20 most retweeted words. Include only words that appear in at least 25 tweets. As usual, try to do this without any for loops. You can string together ~5-7 pandas commands and get everything done on one line.

In [ ]:
#top_20 = ...
### BEGIN SOLUTION
top25 = tmp.groupby('word').filter(lambda x: len(x) >= 25)
merged = top25.merge(df_trump, how='inner', right_index=True, left_index= True)
avg = merged.groupby('word').agg({'retweet_count': 'mean'})
retweeted= avg['retweet_count'].sort_values(ascending=False)
top_20 =  retweeted.iloc[0:20]
top_20
### END SOLUTION

### Task 6.2
Plot a bar chart of your results:

In [ ]:
### BEGIN SOLUTION
top_20.plot.barh(figsize=(10, 10));

### BEGIN SOLUTION

## PART 7 - Conclusion (Optional for Individual)
What else can we do? Let us ask some open ended questions.

### Task 7.1
"kim", "jong" and "un" are apparently really popular in Trump's tweets! It seems like we can conclude that his tweets involving jong are more popular than his other tweets. Or can we?

Consider each of the statements about possible confounding factors below. State whether each statement is true or false and explain. If the statement is true, state whether the confounding factor could have made kim jong un related tweets higher in the list than they should be.

1. We didn't restrict our word list to nouns, so we have unhelpful words like "let" and "any" in our result.
      - That might be why 'un' is the most popular.
1. We didn't remove hashtags in our text, so we have duplicate words (eg. #great and great).
      - Some may only have '#great' not 'great' which make the average lower
1. We didn't account for the fact that Trump's follower count has increased over time.
      - This can affect a lot. As Trump's follower count has increased, the more popular every word be

In [ ]:
#### BEGIN SOLUTION
#1. Although we didnt restrict our word list to nouns, 'un' is not a word by itself so it would only appear as many times as kim jong un
#2. This is true, and might have affected our count of great and other words that have hashtags
#3. This is also may be true, as the more followers, more popular a word may be and may make kim jung un tweets more popular


#### END SOLUTION

### Task 7.2
Using the `df_trump` tweets construct an interesting plot describing a property of the data and discuss what you found below.

**Ideas:**

1. How has the sentiment changed with length of the tweets?
1. Does sentiment affect retweet count?
1. Are retweets more negative than regular tweets?
1. Are there any spikes in the number of retweets and do the correspond to world events? 
1. What terms have an especially positive or negative sentiment?

You can look at other data sources and even tweets. Do some plots and discuss. You can add more cells here as needed.


In [72]:
#### BEGIN SOLUTION
# What terms have an especially positive or negative sentiment?
from textblob import TextBlob

tmp['sentiment'] = tmp['word'].apply(lambda x: TextBlob(x).sentiment.polarity)
tmp.drop_duplicates(subset = 'word', keep = 'first', inplace = True)
sorted_sentiment_negative = tmp.sort_values('sentiment').iloc[0:10]
sorted_sentiment_negative.set_index('word', inplace = True)
ax = sns.barplot(y = sorted_sentiment_negative.index, x ='sentiment', data = sorted_sentiment_negative)
#### END SOLUTION


#### BEGIN SOLUTION
Discussion: "Enter question you tried answering"

What terms have an especially negative sentiment?

Answer:
evil, boring, horribly, horrible, disgusting, outrageous, devastating, terribly, nasty, vicious 
#### END SOLUTION

### Group Part - Find Something interesting (Optional for Individuals)
Is there still something interesting to find in this data set? Use your own imagination to ask some good questions. Don't be bias and look for the answer in data. Don't ask us what we want, because we do not know either. This will be for EXTRA CREDIT for individuals but part of the regular assignment for groups. Add any cells below.


1) Does trump use words that have negative sentiment more often than positive? 
2) Do the tweets with the most retweets contain more negative setiment words?
3) What phrases are frequent in Trump's tweets? 

In [91]:
#2
df_trump['sentiment'] = df_trump['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
# df_trump
most_retweets = df_trump[['sentiment', 'retweet_count']]
most_retweets.sort_values(by = 'retweet_count', inplace = True, ascending = False)
most_retweets.head(20)

#The most retweeted tweets are mostly neutral in sentiment. Positive sentiment is almost lacking, and some of the tweets have a negative sentiment


<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as yourSection_yourNetID_midsemester.jpynb<br>
<b> Group Projects:</b> Each person in the group must submit a copy with both names listed. If you are doing a group project, you must inform your TA prior to 11/3/21 that you intend to work as a group and submit your name and your partner name. We will <b>not accept group work</b> if your TA has not been notified.<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; midsemester (remove all output. Do not submit data files<br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Created by Andy Guna @2019-2021 Credits: Josh Hug, and Berkeley Data Science Group, Steve Skiena, David Rodreguez

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c965daa-37bb-4d88-827a-43213f4176fe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>